In [7]:
from diffusers import DiffusionPipeline
import torch
import numpy as np
import random
from PIL import Image
from diffusers.utils import make_image_grid
import os
import pickle
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import cosine_similarity

Generate Images

In [ ]:
# load both base & refiner
base = DiffusionPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-base-1.0",
    torch_dtype=torch.float16,
    variant="fp16",
    use_safetensors=True,
    cache_dir='/mnt/A-NVME/jordankp/sdxl_models/')
base.to("cuda")
refiner = DiffusionPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-refiner-1.0",
    text_encoder_2=base.text_encoder_2,
    vae=base.vae,
    torch_dtype=torch.float16,
    use_safetensors=True,
    variant="fp16",
    cache_dir='/mnt/A-NVME/jordankp/sdxl_models/')
refiner.to("cuda")
print('done')

In [ ]:
# gets the list of random seeds used in CFG
for folder in os.listdir('/home/jordankp/sdxl/UI_label_images/labeled_images/'):
    for file in os.listdir('/home/jordankp/sdxl/UI_label_images/labeled_images/'+folder):
        #print('/home/jordankp/sdxl/UI_label_images/labeled_images/'+folder+'/'+file)
        with open('/home/jordankp/sdxl/UI_label_images/labeled_images/'+folder+'/'+file, 'rb') as f:
            imgs_list = pickle.load(f)
        
        if 'df' not in globals():
            df = pd.DataFrame(imgs_list)
        else:
            df = pd.concat([df, pd.DataFrame(imgs_list)], axis=0, ignore_index=True)
df.drop('index', axis=1, inplace=True)
df['orig_deb'] = df['img_path'].str.contains('original').replace(True,'original').replace(False,'debiased')

rnd_list = df['rnd'].unique()
rnd_list.shape

In [ ]:
# Define Hyper-Parameters
n_steps = 50
high_noise_frac = 0.99
generation_folder = 'firefighter'
prompt = 'the face of firefighter'
guidance_scale = 12
negative_prompt = 'man traits, very manly male man, rougth man, man, male, man, male, man, male'
prompt_eng = 'a single firefighter person, focused on face, female or male' # best prompt engineering to balance a single person between males and females
lookup_prompt = 'the face of a '+ random.choice(['male','female']) +' firefighter'

all_images = []
num_samples = 2

# selects X number of random seeds
rnd_list = rnd_list[0:num_samples]

for seed in rnd_list:
    for mode in range(5):
        if mode==0: # default prompt
            torch.manual_seed(seed)
            image = base(
                prompt=prompt,
                negative_prompt=None,
                num_inference_steps=n_steps,
                denoising_end=high_noise_frac,
                guidance_scale=guidance_scale,
                output_type="latent"
            ).images
            image_default = refiner(
                prompt=prompt,
                negative_prompt=None,
                num_inference_steps=n_steps,
                denoising_start=high_noise_frac,
                image=image,
                guidance_scale=guidance_scale,
            ).images[0]
        elif mode==1: # negative prompt
            torch.manual_seed(seed)
            image = base(
                prompt=prompt,
                negative_prompt=negative_prompt,
                num_inference_steps=n_steps,
                denoising_end=high_noise_frac,
                guidance_scale=guidance_scale,
                output_type="latent"
            ).images
            image_negative = refiner(
                prompt=prompt,
                negative_prompt=negative_prompt,
                num_inference_steps=n_steps,
                denoising_start=high_noise_frac,
                image=image,
                guidance_scale=guidance_scale,
            ).images[0]
        elif mode==2: # prompt engineering
            torch.manual_seed(seed)
            image = base(
                prompt=prompt_eng,
                negative_prompt=None,
                num_inference_steps=n_steps,
                denoising_end=high_noise_frac,
                guidance_scale=guidance_scale,
                output_type="latent"
            ).images
            image_prompt_eng = refiner(
                prompt=prompt_eng,
                negative_prompt=None,
                num_inference_steps=n_steps,
                denoising_start=high_noise_frac,
                image=image,
                guidance_scale=guidance_scale,
            ).images[0]
        elif mode==3: # Lookup table
            torch.manual_seed(seed)
            image = base(
                prompt=lookup_prompt,
                negative_prompt=None,
                num_inference_steps=n_steps,
                denoising_end=high_noise_frac,
                guidance_scale=guidance_scale,
                output_type="latent"
            ).images
            image_lookup_table = refiner(
                prompt=lookup_prompt,
                negative_prompt=None,
                num_inference_steps=n_steps,
                denoising_start=high_noise_frac,
                image=image,
                guidance_scale=guidance_scale,
            ).images[0]
        elif mode==4:
            image_cfg = Image.open(
                '/home/jordankp/sdxl/UI_label_images/static/images/'+
                generation_folder+'/'+
                generation_folder+'_debiased_'
                +str(seed)+'.jpg')
    all_images.append([seed,image_default,image_cfg,image_negative,image_prompt_eng,image_lookup_table])

Load Images into Grid

In [2]:
# Load list of images from a pickle file
with open('./all_images.pkl', 'rb') as file:
    all_images = pickle.load(file)

Plot Image Grids

In [ ]:
# Plot images on grid for each seed
num_samples = len(all_images)
fig, axs = plt.subplots(num_samples, 1, figsize=(15, num_samples))
axs = axs.flatten()

for i in range(num_samples):
    grid = make_image_grid(all_images[i][1:], rows=1, cols=5, resize=400)
    label = all_images[i][0]
    
    axs[i].imshow(grid)
    axs[i].axis("off")
    axs[i].text(-0.1, 0.5, 'Seed: '+str(label), fontsize=10, va="center", ha="right",
            transform=axs[i].transAxes)
    
plt.tight_layout()
plt.show()

Measures Cosine Similarity between Methods

In [ ]:
def cosine_similarity_between_images(image1, image2):
    img_vector1 = np.array(image1).flatten().reshape(1,-1)
    img_vector2 = np.array(image2).flatten().reshape(1,-1)

    similarity = cosine_similarity(img_vector1, img_vector2)[0, 0]
    return similarity

def cos_sim_on_grid(list_of_images_w_seed):
    cfg_cos_sim = []
    neg_promp_cos_sim = []
    promp_eng_cos_sim = []
    lookup_tbl_cos_sim = []

    for i in range(len(list_of_images_w_seed)):
        cfg_cos_sim.append(cosine_similarity_between_images(list_of_images_w_seed[i][1], list_of_images_w_seed[i][2]))
        neg_promp_cos_sim.append(cosine_similarity_between_images(list_of_images_w_seed[i][1], list_of_images_w_seed[i][3]))
        promp_eng_cos_sim.append(cosine_similarity_between_images(list_of_images_w_seed[i][1], list_of_images_w_seed[i][4]))
        lookup_tbl_cos_sim.append(cosine_similarity_between_images(list_of_images_w_seed[i][1], list_of_images_w_seed[i][5]))

    return cfg_cos_sim, neg_promp_cos_sim, promp_eng_cos_sim, lookup_tbl_cos_sim

In [ ]:
# Similaridade do Cosseno para toda a lista de imagens:
cfg_cos_sim, neg_promp_cos_sim, promp_eng_cos_sim, lookup_tbl_cos_sim = cos_sim_on_grid(all_images)
print(f'''
ClassFreeG: avg= {np.mean(cfg_cos_sim)*100} | std= {np.std(cfg_cos_sim)}
Neg Prompt: avg= {np.mean(neg_promp_cos_sim)*100}  | std= {np.std(neg_promp_cos_sim)}
Prompt Eng: avg= {np.mean(promp_eng_cos_sim)*100} | std= {np.std(promp_eng_cos_sim)}
Lookup Tbl: avg= {np.mean(lookup_tbl_cos_sim)*100} | std= {np.std(lookup_tbl_cos_sim)}''')

Average Image Comparison

In [ ]:
# Average Image of each Method
def avg_images_by_method(list_of_images_w_seed):
    imgs_default = [img[1] for img in list_of_images_w_seed]
    imgs_cfg = [img[2] for img in list_of_images_w_seed]
    imgs_neg_prompt = [img[3] for img in list_of_images_w_seed]
    imgs_prompt_eng = [img[4] for img in list_of_images_w_seed]
    imgs_lookup_tbl = [img[5] for img in list_of_images_w_seed]

    avg_img_default = np.array(imgs_default).sum(axis=0) / np.array(imgs_default).shape[0]
    avg_img_cfg = np.array(imgs_cfg).sum(axis=0) / np.array(imgs_cfg).shape[0]
    avg_img_neg_prompt = np.array(imgs_neg_prompt).sum(axis=0) / np.array(imgs_neg_prompt).shape[0]
    avg_img_prompt_eng = np.array(imgs_prompt_eng).sum(axis=0) / np.array(imgs_prompt_eng).shape[0]
    avg_img_lookup_tbl = np.array(imgs_lookup_tbl).sum(axis=0) / np.array(imgs_lookup_tbl).shape[0]

    return avg_img_default, avg_img_cfg, avg_img_neg_prompt, avg_img_prompt_eng, avg_img_lookup_tbl

In [ ]:
avg_img_default, avg_img_cfg, avg_img_neg_prompt, avg_img_prompt_eng, avg_img_lookup_tbl = avg_images_by_method(all_images)

In [ ]:
print(f'''Cos Sim between Average Image Default vs Average Image of Methods:
ClassFreeG: {cosine_similarity_between_images(avg_img_default, avg_img_cfg)*100}
Neg Prompt: {cosine_similarity_between_images(avg_img_default, avg_img_neg_prompt)*100}
Prompt Eng: {cosine_similarity_between_images(avg_img_default, avg_img_prompt_eng)*100}
Lookup Tbl: {cosine_similarity_between_images(avg_img_default, avg_img_lookup_tbl)*100}''')

In [ ]:
rendered_avg_imgs = []
rendered_avg_imgs.append(Image.fromarray(np.uint8(avg_img_default)))
rendered_avg_imgs.append(Image.fromarray(np.uint8(avg_img_cfg)))
rendered_avg_imgs.append(Image.fromarray(np.uint8(avg_img_neg_prompt)))
rendered_avg_imgs.append(Image.fromarray(np.uint8(avg_img_prompt_eng)))
rendered_avg_imgs.append(Image.fromarray(np.uint8(avg_img_lookup_tbl)))
make_image_grid(rendered_avg_imgs, rows=1, cols=5, resize=200)

Feature Extraction - ResNet-50

In [3]:
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.models as models

In [4]:
def feature_extraction(list_of_images_w_seed):
    # Load pre-trained ResNet model
    resnet_model = models.resnet50(pretrained=True)
    # Remove the classification head (final fully connected layer)
    resnet_model = nn.Sequential(*list(resnet_model.children())[:-1])
    # Set the model to evaluation mode
    resnet_model.eval()
    resnet_model = resnet_model.to('cuda:0')
    print('model done!')

    # Define the image transformation
    # Standard input size for ResNet is (224, 224, 3), and it expects normalized pixel values
    preprocess = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])

    feature_list = []
    for i in range(len(list_of_images_w_seed)):
        # Load and preprocess the image
        input_image_default = preprocess(list_of_images_w_seed[i][1]).unsqueeze(0).to('cuda:0')
        input_image_cfg = preprocess(list_of_images_w_seed[i][2]).unsqueeze(0).to('cuda:0')
        input_image_neg_prompt = preprocess(list_of_images_w_seed[i][3]).unsqueeze(0).to('cuda:0')
        input_image_prompt_eng = preprocess(list_of_images_w_seed[i][4]).unsqueeze(0).to('cuda:0')
        input_image_lookup_tbl = preprocess(list_of_images_w_seed[i][5]).unsqueeze(0).to('cuda:0')
        
        # Use the model to extract image features, transfer back to cpu and flatten vector
        with torch.no_grad():
            features_default = resnet_model(input_image_default).cpu().flatten().reshape(1,-1)
            features_cfg = resnet_model(input_image_cfg).cpu().flatten().reshape(1,-1)
            features_neg_prompt = resnet_model(input_image_neg_prompt).cpu().flatten().reshape(1,-1)
            features_prompt_eng = resnet_model(input_image_prompt_eng).cpu().flatten().reshape(1,-1)
            features_lookup_tbl = resnet_model(input_image_lookup_tbl).cpu().flatten().reshape(1,-1)
        
        feature_list.append([features_default,features_cfg,features_neg_prompt,features_prompt_eng,features_lookup_tbl])

    return feature_list

In [5]:
def cos_sim_of_features(feature_list):
    cs_feat_cfg = []
    cs_feat_neg_promp = []
    cs_feat_promp_eng = []
    cs_feat_lookup_tbl = []

    for i in range(len(feature_list)):
        cs_feat_cfg.append(cosine_similarity(feature_list[i][0], feature_list[i][1])[0, 0])
        cs_feat_neg_promp.append(cosine_similarity(feature_list[i][0], feature_list[i][2])[0, 0])
        cs_feat_promp_eng.append(cosine_similarity(feature_list[i][0], feature_list[i][3])[0, 0])
        cs_feat_lookup_tbl.append(cosine_similarity(feature_list[i][0], feature_list[i][4])[0, 0])

    return cs_feat_cfg, cs_feat_neg_promp, cs_feat_promp_eng, cs_feat_lookup_tbl

In [10]:
# Similaridade do Cosseno para toda a lista de imagens:
feature_list = feature_extraction(all_images)
cs_feat_cfg, cs_feat_neg_promp, cs_feat_promp_eng, cs_feat_lookup_tbl = cos_sim_of_features(feature_list)

print(f'''\nCosine Similarity of Extracted Features between Default and Debias Methods:
ClassFreeG: avg= {np.mean(cs_feat_cfg)*100} | std= {np.std(cs_feat_cfg)}
Neg Prompt: avg= {np.mean(cs_feat_neg_promp)*100} | std= {np.std(cs_feat_neg_promp)}
Prompt Eng: avg= {np.mean(cs_feat_promp_eng)*100} | std= {np.std(cs_feat_promp_eng)}
Lookup Tbl: avg= {np.mean(cs_feat_lookup_tbl)*100} | std= {np.std(cs_feat_lookup_tbl)}''')

/home/jordankp/miniconda3/envs/sdxl/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/jordankp/miniconda3/envs/sdxl/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


model done!

Cosine Similarity of Extracted Features between Default and Debias Methods:
ClassFreeG: avg= 95.66314816474915 | std= 0.014811497181653976
Neg Prompt: avg= 90.95184206962585 | std= 0.047348033636808395
Prompt Eng: avg= 86.88436150550842 | std= 0.06912991404533386
Lookup Tbl: avg= 87.99535036087036 | std= 0.052348311990499496
